In [ ]:
import numpy as np
import pandas as pd 
import os

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/csv_files/reviews.csv')


In [ ]:
data.info()

In [ ]:
df=pd.DataFrame(data)

In [ ]:
df.columns = df.columns.str.replace(' ', '')

In [ ]:
reviews=data[['review.text']]
print(reviews.info())

In [ ]:
reviews.sample(10)

In [ ]:
reviews_text=reviews['review.text'].values.tolist()

In [ ]:
import re
reviews_text=[re.sub(r'[^A-Za-z\s]','',text) for text in reviews_text] 

In [ ]:
from pprint import pprint
pprint(len(reviews_text[0]))

print(reviews_text[0:3])

In [ ]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(texts):
    for text in texts:
        yield(simple_preprocess(str(text),deacc=True))
        
data_words=list(sent_to_words(reviews_text))

In [ ]:

import nltk
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')
new_stopwords = ['movie','movies','film','love','good','like','best','great','ring','shawshank','films','one','ive','watch','thing']
stopword.extend(new_stopwords)

def remove_stopwords(texts):
    return [[word for word in text if word not in stopword] for text in texts]

data_words_nostops=remove_stopwords(data_words)
    



In [ ]:
import gensim

bigram=gensim.models.Phrases(data_words_nostops,min_count=5,threshold=100)
trigram=gensim.models.Phrases(bigram[data_words_nostops],threshold=100)

bigram_mod=gensim.models.phrases.Phraser(bigram)

def make_bigram(texts):
    return [bigram_mod[doc] for doc in texts]

data_words_nostops_bigrams=make_bigram(data_words_nostops)

In [ ]:
import spacy
nlp=spacy.load("en_core_web_sm",disable=['parser','ner'])

def lemmatize(texts,allowed_postags=['NOUN','ADJ','VERB','ADV']):
    texts_out=[]
    for sent in texts:
        doc=nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        
    return texts_out

In [ ]:
data_lemmatized=lemmatize(data_words_nostops_bigrams,allowed_postags=['NOUN','ADJ','VERB','ADV'] )

In [ ]:
import gensim.corpora as corpora
id2word=corpora.Dictionary(data_lemmatized)
texts=data_lemmatized
corpus=[id2word.doc2bow(text) for text in texts]

In [ ]:
print([[(id2word[id],freq) for (id,freq) in cp]for cp in corpus[:2]])

In [ ]:
#Modelleme kısmı
lda_model=gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=9,random_state=100,update_every=1,
                                         chunksize=100,passes=10,alpha='auto',per_word_topics=True)


pprint(lda_model.print_topics())

doc_lda=lda_model[corpus]

In [ ]:
print(list(doc_lda)[0])

In [ ]:
print(lda_model[corpus[0]])
print("*****************")
print("The Topics Distribution for first doc: ")
print(lda_model[corpus[0]][0])

In [ ]:

print("Perplexity: ",lda_model.log_perplexity(corpus))

from gensim.models import CoherenceModel
coherence_model_lda=CoherenceModel(model=lda_model,texts=data_lemmatized,dictionary=id2word,coherence='c_v')

coherence_lda=coherence_model_lda.get_coherence()
print('Coherence Score: ',coherence_lda)

In [ ]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [ ]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)


pyLDAvis.enable_notebook()

vis=pyLDAvis.gensim_models.prepare(lda_model,corpus,id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.322812 -0.005932       1        1  30.211222
8     -0.239351 -0.252635       2        1  22.516409
7     -0.144274  0.370422       3        1  16.925700
6      0.044174 -0.076411       4        1   6.812263
3      0.125149 -0.013711       5        1   6.681646
1      0.098913  0.002973       6        1   5.204810
0      0.131027  0.014080       7        1   4.293018
4      0.148370 -0.016393       8        1   4.155978
2      0.158804 -0.022395       9        1   3.198954, topic_info=            Term           Freq          Total Category  logprob  loglift
145        movie  133075.000000  133075.000000  Default  30.0000  30.0000
45          film   65598.000000   65598.000000  Default  29.0000  29.0000
62          life   22085.000000   22085.000000  Default  28.0000  28.0000
121         good   49308.000000   49308.000000  Default  27.0000  27.0000
564        story   29796.000000   29796.000000  Default  26.0000  26.0000
...          ...            ...            ...      ...      ...      ...
7543        yard     411.733835     412.878306   Topic9  -5.2616   3.4396
300   difference     410.590781     411.733003   Topic9  -5.2644   3.4396
1369       magic     410.292514     411.434734   Topic9  -5.2651   3.4396
2975         fly     402.075891     403.218101   Topic9  -5.2854   3.4395
9932        rape     579.093288     584.207907   Topic9  -4.9205   3.4336

[357 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
987       6  0.995342     ability
711       1  0.999385  absolutely
5712      9  0.998381       abuse
156       1  0.031665         act
156       3  0.968220         act
...     ...       ...         ...
114       2  0.188699       write
114       3  0.786019       write
7543      9  0.997873        yard
282       1  0.011085        year
282       2  0.988787        year

[474 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 9, 8, 7, 4, 2, 1, 5, 3])

In [ ]:
for m, cv in zip(x,coherence_vals):
    print("num topics: ",m,'has coherence value of :',round(cv,4))

num topics:  2 has coherence value of : 0.5476
num topics:  6 has coherence value of : 0.4623
num topics:  10 has coherence value of : 0.3938
num topics:  14 has coherence value of : 0.4016
num topics:  18 has coherence value of : 0.327
